# Load Portfolio Positions

In [1]:
import os
import pandas as pd
import decimal
from instrument_data_access.IEX_dao import *
from instrument_data_access.quandl_dao import *
from instrument_data_access.binance_dao import *
from pathlib import Path, PurePath
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_table as dt


# Constants
BASE_DIRECTORY = PurePath(os.getcwd())
DATA_DIRECTORY = BASE_DIRECTORY.parent / 'data'
PORTFOLIO_POSITIONS_FILE = DATA_DIRECTORY / 'positions.csv'

iex_data_importer = iex_dao()
quandl_data_importer = quandl_dao()
binance_data_importer = binance_dao()

# Load Portfolio Positions
positions_df = pd.read_csv(PORTFOLIO_POSITIONS_FILE.as_posix())
print(positions_df)


     Instrument            Type       Quantity
0          AMZN           Stock      78.000000
1           SPY             ETF      89.000000
2           QQQ             ETF     120.000000
3           VTI             ETF     275.000000
4            VT             ETF      49.000000
5           VEA             ETF     379.000000
6           EEM             ETF     187.000000
7           VWO             ETF     389.000000
8          MCHI             ETF     225.000000
9           BWX             ETF     220.000000
10          DBC             ETF    1557.000000
11    LBMA/GOLD       Commodity      20.000000
12  LBMA/SILVER       Commodity    2755.000000
13          USD        Currency  156790.850000
14      BTCUSDT  Cryptocurrency       1.253041
15      ETHUSDT  Cryptocurrency      19.391493
16      XRPUSDT  Cryptocurrency    3103.570000
17      BCHUSDT  Cryptocurrency       1.771802
18      LTCUSDT  Cryptocurrency      25.429680
19     NANOUSDT  Cryptocurrency     149.980000
20      DOTUS

# Filter Equities

In [2]:

index_list = []
for row_index, row in positions_df.iterrows():
    if(row['Type'] == "ETF") or (row['Type'] == 'Stock'):
        index_list.append(row['Instrument'])
print(index_list)
index_pd = pd.DataFrame(index_list, columns=['Instrument'])
print(index_pd)

['AMZN', 'SPY', 'QQQ', 'VTI', 'VT', 'VEA', 'EEM', 'VWO', 'MCHI', 'BWX', 'DBC']
   Instrument
0        AMZN
1         SPY
2         QQQ
3         VTI
4          VT
5         VEA
6         EEM
7         VWO
8        MCHI
9         BWX
10        DBC


# Get Last Prices for Equities

In [10]:
last_price_df = pd.DataFrame()

for row_index, row in positions_df.iterrows():
    if(row['Type'] == "ETF") or (row['Type'] == 'Stock'):
        last_price = float(iex_data_importer.get_last_price(row['Instrument']).iloc[0,0])
        df = {'Last_Price': last_price}
        last_price_df = last_price_df.append(df, ignore_index=True)
    elif (row['Type'] == "Commodity"):
        last_price = float(quandl_data_importer.get_latest_instrument_price(row['Instrument']).iloc[0,0])
        df = {'Last_Price': last_price}
        last_price_df = last_price_df.append(df, ignore_index=True)
    elif (row['Type'] == "Cryptocurrency"):
        last_price = float(binance_data_importer.get_last_price(row['Instrument']))
        df = {'Last_Price': last_price}
        last_price_df = last_price_df.append(df, ignore_index=True)
    elif (row['Type'] == "Currency"):
        df = {'Last_Price': 1.0}
        last_price_df = last_price_df.append(df, ignore_index=True)
# TODO: Add an index and merge data frames together
print(last_price_df)

positions_df = positions_df.assign(Last_Price=last_price_df)
positions_df

IEX Processing: AMZN
IEX Processing: SPY
IEX Processing: QQQ
IEX Processing: VTI
IEX Processing: VT
IEX Processing: VEA
IEX Processing: EEM
IEX Processing: VWO
IEX Processing: MCHI
IEX Processing: BWX
IEX Processing: DBC
Quandle Processing: LBMA/GOLD
Quandle Processing: LBMA/SILVER
Binance processing: BTCUSDT
Binance processing: ETHUSDT
Binance processing: XRPUSDT
Binance processing: BCHUSDT
Binance processing: LTCUSDT
Binance processing: NANOUSDT
Binance processing: DOTUSDT
    Last_Price
0    3401.4600
1     426.6100
2     349.4600
3     222.5500
4     103.9800
5      52.2200
6      55.5000
7      54.5600
8      83.0200
9      29.4400
10     18.9900
11   1783.2500
12     26.1300
13      1.0000
14  34330.0000
15   1947.5200
16      0.6410
17    471.8600
18    130.9400
19      4.8326
20     14.8260


,Instrument,Type,Quantity,Last_Price
0,AMZN,Stock,78.000000,3401.4600
1,SPY,ETF,89.000000,426.6100
2,QQQ,ETF,120.000000,349.4600
3,VTI,ETF,275.000000,222.5500
4,VT,ETF,49.000000,103.9800
5,VEA,ETF,379.000000,52.2200
6,EEM,ETF,187.000000,55.5000
7,VWO,ETF,389.000000,54.5600
8,MCHI,ETF,225.000000,83.0200
9,BWX,ETF,220.000000,29.4400


# Calculate Total Assets

In [11]:

def currency_format(x):
    return "${0:,.2f}".format(x)

positions_df['Total_Assets'] = positions_df['Quantity'].astype(int) * positions_df['Last_Price'].astype(float)
positions_df['Total_Assets'] = positions_df['Total_Assets'].apply(currency_format)
positions_df['Last_Price'] = positions_df['Last_Price'].astype(float).apply(currency_format)
positions_df


,Instrument,Type,Quantity,Last_Price,Total_Assets
0,AMZN,Stock,78.000000,"$3,401.46","$265,313.88"
1,SPY,ETF,89.000000,$426.61,"$37,968.29"
2,QQQ,ETF,120.000000,$349.46,"$41,935.20"
3,VTI,ETF,275.000000,$222.55,"$61,201.25"
4,VT,ETF,49.000000,$103.98,"$5,095.02"
5,VEA,ETF,379.000000,$52.22,"$19,791.38"
6,EEM,ETF,187.000000,$55.50,"$10,378.50"
7,VWO,ETF,389.000000,$54.56,"$21,223.84"
8,MCHI,ETF,225.000000,$83.02,"$18,679.50"
9,BWX,ETF,220.000000,$29.44,"$6,476.80"


# Calculate Total Liquid Net Worth

In [16]:
total_liquid_net_worth = (positions_df['Total_Assets'].replace('[\$,)]','', regex=True)
                            .replace('[(]','-', regex=True).astype(float)).sum()
print('${:,.2f}'.format(total_liquid_net_worth))

$860,217.38


In [5]:

instruments_info_dict = {}

for row_index, row in positions_df.iterrows():
    if (row['Type'] == "ETF") or (row['Type'] == 'Stock'):
        instrument = row['Instrument']
        instrument_info_dict = iex_data_importer.get_company_info(instrument)
        instruments_info_dict[instrument] = instrument_info_dict
    


print(instruments_info_dict)

{'AMZN': {'symbol': 'AMZN', 'companyName': 'Amazon.com, Inc.', 'exchange': 'NASDAQ', 'industry': 'Internet Retail', 'website': 'http://www.amazon.com', 'description': 'Amazon.com, Inc. engages in the provision of online retail shopping services. It operates through the following business segments: North America, International, and Amazon Web Services (AWS). The North America segment includes retail sales of consumer products and subscriptions through North America-focused websites such as www.amazon.com and www.amazon.ca. The International segment offers retail sales of consumer products and subscriptions through internationally-focused websites. The Amazon Web Services segment involves in the global sales of compute, storage, database, and AWS service offerings for start-ups, enterprises, government agencies, and academic institutions. The company was founded by Jeffrey P. Bezos in July 1994 and is headquartered in Seattle, WA.', 'CEO': 'Jeffrey P. Bezos', 'securityName': 'Amazon.com,